In [1]:
from __future__ import division
import gym
import torch
import random
import numpy as np
import torch
from PIL import Image
import torch
import torch.nn as nn
from collections import namedtuple
from collections import deque
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm
from matplotlib.pyplot import imshow
from PIL import Image
from wrappers import make_atari, wrap_deepmind, wrap_pytorch
import queue
from torch import optim
import matplotlib.pyplot as plt

# Model

In [2]:
class QNet(torch.nn.Module):
    def __init__(self,obs_shape,act_shape,atoms):
        super(QNet, self).__init__()
        self.atoms = atoms
        self.act_shape = act_shape
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.relu = nn.ReLU()
        #Linear layers for dueling
        self.fc1 = nn.Linear(7*7*64,512)
        self.fc2 = nn.Linear(512,atoms)
        self.fc3 = nn.Linear(7*7*64,512)
        self.fc4 = nn.Linear(512,act_shape*atoms)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x=x/255
        #Conv
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = x.view(x.shape[0],-1)
        
        #Fc
        x1 = x
        x = self.fc1(x)
        x = self.relu(x)
        v = self.fc2(x)
        
        x1 = self.fc3(x1)
        x1 = self.relu(x1)
        adv = self.fc4(x1)
        
        #Reshaping value and advantage functions to add probabilities of each atom for each action
        value = v.view(v.shape[0],1,self.atoms)
        adv = adv.view(adv.shape[0],self.act_shape,self.atoms)
        
        q_s_a = value + adv - adv.mean(1,keepdim=True)
        
        #probability of each atom for all actions
        q_s_a = self.softmax(q_s_a)
        
        return q_s_a

In [3]:
def eps_greedy(epsilon,state,net,atoms):
    if(np.random.random()<epsilon):
        action = np.random.randint(ACT_SHAPE)
    else:
        #Finding the expected value of each action (sum(pi*zi))
        qvalues = net(state)
        expected_values = torch.matmul(qvalues,atoms)
        action = torch.argmax(expected_values).item()
    return action    

In [4]:
class ReplayBuffer(object):
    def __init__(self,maxsize):
        self.q = deque(maxlen = maxsize)
        self.maxsize = maxsize
    def add(self,x):
        self.q.append(x)
        if(len(self.q)==self.maxsize):
            self.q.popleft()
    def getSize(self):
        return len(self.q)
    def sample(self,size):
        batch = random.sample(list(self.q),size)
        state,action,reward,next_state,done = map(list, zip(*batch))
        return state,action,reward,next_state,done
           

In [5]:
class SegmentSumTree:
    def __init__(self, capacity):
        self.size = capacity 
        #n leaves + n-1 internal = 2n-1, capacity of a tree
        self.tree = np.zeros((2*capacity-1), dtype = np.float32)
        #leaf nodes having actual values 
        self.transit_data_buffer =  np.array([None] * self.size) 
        self.is_full = False
        self.max_prior_val = 1
        self.num_entries = 0 
        self.idx = 0 
    
    
    def append(self, priority_val, transit_data):
        #self.max_prior_val = max(priority_val, self.max_prior_val)
        self.transit_data_buffer[self.idx] = transit_data  
        self.update(self.idx + self.size - 1, priority_val) 
        self.idx = (self.idx + 1) % self.size 
        self.num_entries += 1
        if self.num_entries > self.size:
            self.num_entries = self.size
        self.is_full = self.is_full or self.idx == 0  
    
    
    def propagate(self, index, priority_val):
        parent_idx = self.getParentIdx(index)
        left_node_idx = 2 * parent_idx + 1
        right_node_idx = 2 * parent_idx + 2
        self.tree[parent_idx] = self.tree[left_node_idx] + self.tree[right_node_idx]
        if parent_idx != 0: 
            self.propagate(parent_idx, priority_val)
            
        
    def update(self, index, priority_val):
#         print('priority',priority_val)
        self.tree[index] = priority_val
        self.max_prior_val = max(priority_val, self.max_prior_val)
        self.propagate(index, priority_val)
       
    
    def getParentIdx(self, index):
        return (index - 1)//2
        
            
        
    def search(self, value):
        idx = self.retrieve(0, value)  # Search for index of item from root
        data_index = idx - self.size + 1
        return (self.tree[idx], data_index, idx)
        
    
    def retrieve(self, idx, value):
        left_node_idx = 2 * idx + 1
        right_node_idx = 2 * idx + 2
        
        if left_node_idx >= len(self.tree):
            return idx
        
        elif value <= self.tree[left_node_idx]:
            return self.retrieve(left_node_idx, value)
            
        else:
            return self.retrieve(right_node_idx, value - self.tree[left_node_idx])
        
    
    def getNumEntries(self):
        return self.num_entries
     

    def getTotal(self):
        return self.tree[0]
        
    def getSize(self):
        return self.size
    
    
    def getMaxPriorValue(self):
        return self.max_prior_val
    
    
    def getPriorties(self):
        return self.priorities
    
    
    def getTree(self):
        return self.tree
    
    
    def getDataByIdx(self, idx):
        return self.transit_data_buffer[idx % self.size]
    
    
Transition = namedtuple('Transition', ('state', 'action', 'reward', 'next_state','done'))

class PrioritizedReplayBuffer:
    def __init__(self,capacity = 1000000, batch_size = 32, gamma = 0.99, multi_step = 3, 
                beta = 0.4, alpha = 0.6):
        self.capacity = capacity
        self.discount = gamma
        self.n = multi_step
        self.beta = beta   
        self.alpha = alpha
        self.transit_buffer = SegmentSumTree(capacity) 
               

    def add(self,state,action,reward,next_state,done):
        
        self.transit_buffer.append(self.transit_buffer.max_prior_val, Transition(state, action, reward, next_state, done))  
        
        
    def sample(self,k): 
        batch, idxs, priorities = [], [], []
        root_total_priority = self.transit_buffer.getTotal()
        segment = root_total_priority / k
        
#         priority_exponent_weight = (1 - self.beta) / (TMAX - TMIN)
        priority_exponent_weight = (1 - self.beta) / 1000
        self.beta = np.min([1.0, self.beta + priority_exponent_weight])
#         print('total priority',root_total_priority)
        
        for i in range(k):
            a = segment * i
            b = segment * (i + 1)
            samp = random.uniform(a, b)
#             print('sample : ',samp)
            priority, data_idx, idx = self.transit_buffer.search(samp)
#             print('data idx',data_idx)
            data = self.transit_buffer.getDataByIdx(data_idx)
            priorities.append(priority)
            batch.append(data)
            idxs.append(idx)
        #print('*'*100)
        sum_priority = np.power(self.transit_buffer.tree[:self.transit_buffer.getNumEntries()],self.alpha)
        sum_priority = sum_priority.sum()
        sampling_probabilities = np.power(priorities, self.alpha) / sum_priority #root_total_priority
        #
        
        #print(sampling_probabilities)
        
        #compute importance sampling 
        weights = np.power(self.transit_buffer.getNumEntries() * sampling_probabilities, -self.beta)
        weights /= weights.max()
        
#         try:
        states, actions, rewards, next_states, dones = zip(*batch)
#         except:
#             print('error in ',batch)
        
        states,actions, rewards, next_states, dones = list(states), list(actions), list(rewards), \
                                                    list(next_states), list(dones) 
        states = torch.cat([x for x in states]).cuda()
        next_states = torch.cat([x for x in next_states]).cuda()
        actions = torch.Tensor(actions).long().cuda()
        dones = np.array(dones).astype(int)
        dones = torch.Tensor(dones).cuda()
        rewards = torch.Tensor(rewards).cuda()
        #priorities = torch.Tensor(priorities).cuda()
        weights = torch.Tensor(weights).cuda()
        
        return weights, idxs, states, actions, rewards, next_states, dones       
        
        
    def updatePriorities(self, indexes, priorities):
        #priorities = np.power(priorities, self.alpha)
        for idx, priority_val in zip(indexes, priorities):
            self.transit_buffer.update(idx, priority_val)
        
        
    def getSize(self):
        return self.transit_buffer.getNumEntries()
        

# Loss function

In [6]:
def compute_loss(size):
    weights,idx,current_state,action,reward,next_state,done = buffer.sample(size)
    
    #qvalues -> (batch,actions,atoms)
    # z(xt)
    dist_current = net(current_state)
    
    # z(xt+1)
    dist_next = net(next_state)
    target_net.eval()
    
    # z'(xt+1)
    dist_target = target_net(next_state)
    
    #Selecting optimal action a*
    
    zvalues_next = torch.matmul(dist_next,atoms)
    optimal_action = zvalues_next.max(1)[1]
    
    #Distribution of target with optimal action z(xt+1,a*)
    dist_target_optimal = dist_target.gather(1,optimal_action.view(-1,1).unsqueeze(2).repeat(1,1,N_ATOMS))
    dist_target_optimal = dist_target_optimal.squeeze(1)
    
    #Finding target distribution values (Tzj = r + gamma*Z(x,a*)) (not aligned)
    done = done.squeeze(0)
    done = done.unsqueeze(1)
    Tz = reward.unsqueeze(1).repeat(1,N_ATOMS) + (1-done).repeat(1,N_ATOMS)*GAMMA*atoms.unsqueeze(0).repeat(size,1)
    
    #Clipping the values
    Tz = torch.clamp(Tz,min=VMIN,max=VMAX)
    
    #Aligning the values
    
    deltaz = (VMAX-VMIN)/(N_ATOMS-1)
    indices = (Tz - VMIN)/deltaz
    lower = indices.floor().long()
    upper = indices.ceil().long()
    lower[(upper > 0) * (lower == upper)] -= 1
    upper[(lower < (N_ATOMS - 1)) * (lower == upper)] += 1
    
    #Finding target probabilities
    target_distribution = torch.zeros(size,N_ATOMS).cuda()
    
    offset = torch.linspace(0, ((size - 1) * N_ATOMS), size).unsqueeze(1).expand(size, N_ATOMS).to(action).cuda()
    
    target_distribution = target_distribution.view(-1).index_add_(0, (lower + offset).view(-1), (dist_target_optimal * (upper.float() - indices)).view(-1))  # m_l = m_l + p(s_t+n, a*)(u - b)
    target_distribution = target_distribution.view(-1).index_add_(0, (upper + offset).view(-1), (dist_target_optimal* (indices - lower.float())).view(-1))  # m_u = m_u + p(s_t+n, a*)(b - l) 
    
    #p(xt,at)
    current_distribution = dist_current.gather(1,action.view(-1,1).unsqueeze(2).repeat(1,1,N_ATOMS))
    current_distribution = current_distribution.squeeze(1)
    
    
    target_distribution = torch.reshape(target_distribution,(size,N_ATOMS))
    
    
    L = (-target_distribution*torch.log(current_distribution)).sum(dim=1)
    
    new_priorities =  torch.abs(L) + edge_epsilon
    
    
    L = (L*weights).mean()
    

    new_priorities = new_priorities.cpu().detach().numpy()
    buffer.updatePriorities(idx,new_priorities)

    optimizer.zero_grad()
    L.backward()
    optimizer.step()
    
    
    return L

In [7]:
def update_target():
    target_net.load_state_dict(net.state_dict())

In [8]:
def epsilon_decay(ep):
    e = .01 + .99*np.exp(-ep/30000)
    return e

In [9]:
def addreward(id,item,filename):
    f=open(filename,'a+')
    f.write(str(id)+' '+str(item)+' '+'\n')
    f.close()

In [10]:
def addloss(id,loss,filename):
    f=open(filename,'a+')
    f.write(str(id)+' '+str(loss.item())+' '+'\n')
    f.close()

# Training

In [11]:
# env = gym.make('PongNoFrameskip-v4')
env    = make_atari('PongNoFrameskip-v4')
# env    = make_atari('BankHeist-v0')
# env    = make_atari('Pong-v0')
env    = wrap_deepmind(env)
env    = wrap_pytorch(env)

print(env.observation_space)
print(env.action_space)


Box(4, 84, 84)
Discrete(6)


In [12]:
ITERATIONS = 1000000
epsilon = 0.99
OBS_SHAPE = env.observation_space.shape
ACT_SHAPE = env.action_space.n
REPLAY_SAMPLE = 20000
BATCH_SIZE = 32
GAMMA = 0.99
T_upd = 15000
TMAX = 50e6
TMIN = 20e3
edge_epsilon = 1e-5
N_ATOMS = 51
VMAX = 10
VMIN = -10
atoms = torch.linspace(VMIN,VMAX,N_ATOMS).cuda()

In [13]:
torch.manual_seed(108) 
np.random.seed(108)
net = QNet(env.observation_space.shape,env.action_space.n,N_ATOMS)
net = net.cuda()
target_net = QNet(env.observation_space.shape,env.action_space.n,N_ATOMS)
target_net = target_net.cuda()
update_target()
optimizer = optim.Adam(net.parameters(), lr=0.0000625,eps=1.5e-4)


In [14]:
#buffer = ReplayBuffer(10000)
buffer = PrioritizedReplayBuffer(100000)
episode_reward = 0
state = env.reset()
state = torch.Tensor(state).cuda()
state = state.unsqueeze(0)
count = 0
# lossfile = './dueling-logs/losses.txt'
# rewardsfile = './dueling-logs/rewards.txt'
losses = []
rewards = [] 
for i in tqdm(range(ITERATIONS)):
    epsilon = epsilon_decay(i)
    action = eps_greedy(epsilon,state,net,atoms)
    next_state, reward, done, info = env.step(action)
    next_state = torch.Tensor(next_state).unsqueeze(0).cuda()
#     print('state input in buffer ',state.shape)
#     print('state input in buffer ',next_state.shape)
    episode_reward+=reward
#     reward = max(min(reward,1),-1)
    buffer.add(state,action,reward,next_state,done)
    state=next_state
    if(buffer.getSize()>REPLAY_SAMPLE and i%4==0):
        loss = compute_loss(BATCH_SIZE)
#         losses.append(loss)
#         if(i%100==0):
#             print("Loss",loss.item())
#             addloss(i,loss,lossfile)
    if done:
        count+=1
        rewards.append(episode_reward)
        print(count,episode_reward)
#         if(count%5==0):
#             addreward(count,episode_reward,rewardsfile)
#             print(episode_reward)
        episode_reward = 0 
        state = env.reset()
        state = torch.Tensor(state).cuda()
        state = state.unsqueeze(0)
    if(i%T_upd==0):
#         torch.save(net.state_dict(),'./dueling-logs/dqn-model.pth')
#         torch.save(target_net.state_dict(),'./dueling-logs/dqn-model-target.pth')
        update_target()


/home/ishaan16238/.conda/envs/ishaan/lib/python3.7/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


1 -21.0
2 -20.0
3 -21.0
4 -20.0
5 -21.0
6 -21.0
7 -21.0
8 -21.0
9 -21.0
10 -21.0
11 -21.0
12 -19.0
13 -20.0
14 -21.0
15 -19.0
16 -20.0
17 -21.0
18 -21.0
19 -21.0
20 -20.0
21 -21.0
22 -21.0
23 -21.0
24 -20.0
25 -21.0
26 -21.0
27 -20.0
28 -20.0
29 -21.0
30 -20.0
31 -21.0
32 -21.0
33 -21.0
34 -21.0
35 -20.0
36 -21.0
37 -21.0
38 -21.0
39 -21.0
40 -20.0
41 -20.0
42 -20.0
43 -21.0
44 -21.0
45 -21.0
46 -20.0
47 -18.0
48 -21.0
49 -21.0
50 -20.0
51 -21.0
52 -21.0
53 -21.0
54 -21.0
55 -21.0
56 -20.0
57 -20.0
58 -20.0
59 -21.0
60 -21.0
61 -21.0
62 -21.0
63 -21.0
64 -21.0
65 -21.0
66 -21.0
67 -20.0
68 -21.0
69 -20.0
70 -21.0
71 -20.0
72 -21.0
73 -21.0
74 -21.0
75 -20.0
76 -20.0
77 -21.0
78 -20.0
79 -21.0
80 -21.0
81 -21.0
82 -21.0
83 -20.0
84 -21.0
85 -21.0
86 -21.0
87 -21.0
88 -21.0
89 -21.0
90 -21.0
91 -21.0
92 -21.0
93 -21.0
94 -21.0
95 -20.0
96 -20.0
97 -21.0
98 -21.0
99 -21.0
100 -21.0
101 -21.0
102 -20.0
103 -21.0
104 -21.0
105 -21.0
106 -21.0
107 -21.0
108 -21.0
109 -21.0
110 -21.0
111 -21.

KeyboardInterrupt: 